In [1]:
from app.Data_ETL_Pipeline import Data_ETL_Pipeline
from app.common.MySQLUtility import MySQLUtility
import os 

domains = ['esg', 'liabilities' ] #'liabilities', 'esg'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/gcp/genuine-wording-key.json'

DB_HOST = '34.170.168.203'
DB_USER = 'root'
DB_PASSWORD = 'nu123456'
DB_NAME = 'lca_dev'
mode = 'accuracy'

dbutil = MySQLUtility(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

dbloader = Data_ETL_Pipeline(dbutil, domains, mode)
dbloader.create_dataset()
dbloader.load_file_data()
dbloader.process_seed_training_data() 
dbloader.process_keyword_model()
dbloader.process_transformer_model()
dbloader.evaluate_results()


/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dbutil.db_cleanup():
DB Pool Created.
Table contract_data successfully deleted.
Table seed_data successfully deleted.
Table training_data successfully deleted.
dbutil.create_database():
Table contract_data successfully created.
Table seed_data successfully created.
Table training_data successfully created.
data_load.import_seed_data_batch():
Working with  seed_data_FIN v2.0.csv
INSERT INTO seed_data (id, created, keywords, content, type, label, domain, userid) VALUES ('7eed32ee-bdf8-4f27-b455-c0406900a2d4', '2022-12-09 00:01:30', 'contingent liability, billion liability, company, dollar, million dollar, emission scandal, skeptical eye, billion, volkswagen, contingent liability example, million, specific situation', 'Contingent liabilities should be analyzed with a serious and skeptical eye, since, depending on the specific situation, they can sometimes cost a company several millions of dollars. Sometimes contingent liabilities can arise suddenly and be completely unforeseen. The $4.3 

ERROR:root:Data Empty
NoneType: None
ERROR:root:Data is empty


class_service.training():esg
risk_class.process_keyword_polarity(): esg
Query:  SELECT * from training_data where domain='esg';
Polarity : 
 {'esg': {'keywords': []}}
class_service.prepare_train_dataset():liabilities
Query:  SELECT * from training_data where type='seed' and domain='liabilities';
Query:  SELECT * from training_data where type='seed' and domain='liabilities';
class_service.training():liabilities


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

...

Metrics :  {'eval_loss': 0.29406601190567017, 'eval_runtime': 34.5215, 'eval_samples_per_second': 7.329, 'eval_steps_per_second': 0.927, 'epoch': 3.0}


Model weights saved in ./model/transform/liabilities/pytorch_model.bin


risk_class.process_keyword_polarity(): liabilities
SELECT * from seed_data where domain='liabilities';
Query:  SELECT * from training_data where domain='liabilities';


Disabling tokenizer parallelism, we're using DataLoader multithreading already


Polarity : 
 {'esg': {'keywords': []}, 'liabilities': {'keywords': [{'name': 'contingent liability', 'score': -9174, 'count': 152, 'polarity': -60}, {'name': 'company', 'score': -16227, 'count': 309, 'polarity': -52}, {'name': 'current liability', 'score': -8887, 'count': 190, 'polarity': -46}, {'name': 'example', 'score': -6425, 'count': 112, 'polarity': -57}, {'name': 'liability', 'score': -26874, 'count': 495, 'polarity': -54}, {'name': 'account', 'score': -7638, 'count': 141, 'polarity': -54}, {'name': 'balance sheet', 'score': -3109, 'count': 67, 'polarity': -46}, {'name': 'business', 'score': -2437, 'count': 59, 'polarity': 0}, {'name': 'current asset', 'score': -378, 'count': 34, 'polarity': -11}, {'name': 'financial statement', 'score': -1185, 'count': 38, 'polarity': -31}, {'name': 'payment', 'score': -4449, 'count': 70, 'polarity': -63}, {'name': 'period', 'score': -3591, 'count': 53, 'polarity': -67}, {'name': 'lawsuit', 'score': -3555, 'count': 37, 'polarity': -96}, {'name'

In [2]:
from app.common.GCP_Storage import GCP_Storage

import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/gcp/genuine-wording-key.json'

domains = ['liabilities', 'esg']
bucket_name = 'lca_dev'
loader = GCP_Storage(domains, bucket_name)

#loader.setup_bucket()
#loader.upload_models()
#loader.download_models()
#loader.download_seed_data()
#loader.upload_seed_data()